In [59]:
from circleci.api import Api
import pprint
from collections import defaultdict

token = open(".env").readlines()[0].split("=")[1].strip()
circleci = Api(token)

# get info about your user
#pprint.pprint(circleci.get_user_info())

# get list of all of your projects
# --> build_num --> get_build_info() -> steps -> output_url -> fetch it -> x[0]["message"] is newline delim string
# --> .outcome == "failed" e.g.

results = defaultdict(lambda: defaultdict(int))

# get results 100 at a time

get_how_many = 1000
per_page = 100
pages = int(get_how_many / per_page)

for i in range(pages):
    for build in circleci.get_project_build_summary(
            "pachyderm", "pachyderm-ci-experiment", limit=100, offset=i*per_page,
        ):
        outcome = build["outcome"]
        build_num = build["build_num"]

        if not outcome == "failed" and not outcome == "success":
            continue

        job = build["workflows"]["job_name"]
        #print(f"build {build_num} {outcome} {job}")
        if job.startswith("test-"):
            results[job][outcome] += 1

# Finding flakiest test suites

The following produces a table of test suites ordered by flakiest suite first.

In [61]:
import pandas as pd
df = pd.DataFrame.from_dict(results)

# Transpose
df = df.T

# NaN -> 0
df = df.fillna(0)

df['pass_rate'] = df['success'] / (df['failed'] + df['success'])

# sort by failed
df = df.sort_values(by=["pass_rate"])

df

,failed,success,pass_rate
test-AUTH2,69.0,0.0,0.000000
test-MISC,69.0,0.0,0.000000
test-PFS,68.0,0.0,0.000000
test-EXAMPLES,66.0,0.0,0.000000
test-ADMIN,70.0,0.0,0.000000
test-PPS6,58.0,10.0,0.147059
test-PPS4,53.0,17.0,0.242857
test-PPS1,47.0,23.0,0.328571
test-PPS3,44.0,26.0,0.371429
test-PPS5,43.0,26.0,0.376812


# Finding flakiest individual tests
Now we fetch the logs for each individual test and find the flakiest individual tests.